<a href="https://colab.research.google.com/github/Dheerajthummala/chatbot_TF/blob/main/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [6]:
with open('/content/intents.json') as file:
  data=json.load(file)


In [7]:
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'], 'responses': ['Hello', 'Hi', 'Hi there']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?'], 'responses': ['I.m Joana, your bot assistant', "I'm Joana, an Artificial Intelligent bot"]}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Joana.', "I'm Joana!", 'Just call me as Joana']}, {'tag': 'help', 'patterns': ['Could you help me?', 'give me a hand please', 'Can you help?', 'What can you do for me?', 'I need a support', 'I need a help', 'support me please'], 'resp

In [21]:
train_sentences = []
train_labels = []
responses = []
labels = []

In [22]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        train_sentences.append(pattern)
        train_labels.append(intent['tag'])
        responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [23]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)

In [24]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

sequences = tokenizer.texts_to_sequences(train_sentences)


max_length = 20
padded_sequences = pad_sequences(sequences, padding='post', maxlen=max_length)

vocab_size = len(tokenizer.word_index) + 1

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, 16, input_length=20))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

train_labels = np.array(train_labels)


epochs = 500
history = model.fit(padded_sequences, train_labels, epochs=epochs, batch_size=2)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 16)            848       
                                                                 
 global_average_pooling1d_7  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_15 (Dense)            (None, 16)                272       
                                                                 
 dense_16 (Dense)            (None, 8)                 136       
                                                                 
Total params: 1256 (4.91 KB)
Trainable params: 1256 (4.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500
17/17 [==============================] - 1s 2ms/step - loss: 2.0818 - accura

In [32]:
model.save("D")
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the label encoder
with open('label_encoder.pickle', 'wb') as enc:
    pickle.dump(label_encoder, enc, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
!pip install colorama
import colorama
colorama.init()
from colorama import Fore, Style, Back
import random


In [35]:
with open('/content/intents.json') as file:
  data1=json.load(file)
  def chat():

    model = keras.models.load_model('D')

    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)



In [38]:
max_len = 20
while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = label_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

User: hello
1/1 [==============================] - 0s 115ms/step
ChatBot: Hi
User: name
1/1 [==============================] - 0s 25ms/step
ChatBot: Hi there
User: thank
1/1 [==============================] - 0s 22ms/step
ChatBot: My pleasure
User: name
1/1 [==============================] - 0s 20ms/step
ChatBot: Hi
User: what is your name
1/1 [==============================] - 0s 21ms/step
ChatBot: I'm Joana!
User: 

KeyboardInterrupt: Interrupted by user